# H3: Amount vs Amount_USD

**H3 (Tech):** Конвертация сумм в USD улучшает стабильность модели и переносимость порогов.


In [1]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [2]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features
from src.validation import split_time_aware
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack

DATA=Path('../data'); TX=DATA/'transaction_fraud_data.parquet'; FX=DATA/'historical_currency_exchange.parquet'
df_raw = load_transactions(TX); fx = load_fx(FX)
df_usd = convert_to_usd(df_raw, fx); df_usd = unpack_last_hour_activity(df_usd); df_usd = add_basic_time_features(df_usd)

def train_eval(df_in, use_usd=True):
    cols = df_in.columns.tolist()
    keep = [c for c in cols if c != ('amount_usd' if not use_usd else 'amount')]
    d = df_in[keep].copy()
    train, test = split_time_aware(d)
    y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
    X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])
    pipe = Pipeline([('prep', build_preprocessor(X_tr)), ('clf', build_logreg())]).fit(X_tr, y_tr)
    p = pipe.predict_proba(X_te)[:,1]
    return y_te, p

yA, pA = train_eval(df_usd, use_usd=False)  # amount
yB, pB = train_eval(df_usd, use_usd=True)   # amount_usd

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ wei

In [3]:
print(f'amount: {eval_pack(yA, pA)}')
print(f'amount_usd: { eval_pack(yB, pB)}')

amount: {'roc_auc': 0.9031792506689108, 'pr_auc': 0.8419896250739393, 'threshold_at_precision': 0.9, 'thr_value': 0.961925570155691, 'recall_at_precision': 0.6503379745810735, 'precision_achieved': 0.9}
amount_usd: {'roc_auc': 0.9163088167066666, 'pr_auc': 0.8609266057049232, 'threshold_at_precision': 0.9, 'thr_value': 0.09913382659583368, 'recall_at_precision': 0.6920653552443401, 'precision_achieved': 0.9000021932229411}


Ниже — краткий вывод и сразу практические улучшения эксперимента.

Результат H3 (итоги)
	•	Качество ↑: ROC AUC +1.31 п.п. (0.9032 → 0.9163), PR AUC +1.89 п.п. (0.8420 → 0.8609).
	•	Операционная метрика ↑: Recall@P=0.90 +4.17 п.п. (0.6503 → 0.6921).
	•	Порог вероятности резко «сдвинулся» (0.962 → 0.099), что подчёркивает: фиксировать числовой порог нельзя; переносить следует целевую точность (например, P=0.90) после калибровки.

Вывод: H3 подтверждается по качеству; для заявления о «переносимости порога» нужна проверка стабильности целевой точки (P=0.90) во времени/валютах.